In [1]:
#Load packages
import pandas as pd
import numpy as np

# I. Comparing Event Run Values

In [2]:
# Read in MLBAM Data for 2014-2017

MLBAM14 = pd.read_csv("D:\\GitHub\\Coursera\\SportsPerformanceAnalytics\\MoneyBallAndBeyond\\data\\MLBAM14.csv")
MLBAM15 = pd.read_csv("D:\\GitHub\\Coursera\\SportsPerformanceAnalytics\\MoneyBallAndBeyond\\data\\MLBAM15.csv")
MLBAM16 = pd.read_csv("D:\\GitHub\\Coursera\\SportsPerformanceAnalytics\\MoneyBallAndBeyond\\data\\MLBAM16.csv")
MLBAM17 = pd.read_csv("D:\\GitHub\\Coursera\\SportsPerformanceAnalytics\\MoneyBallAndBeyond\\data\\MLBAM17.csv")

In [3]:
#Your Code Here
def Run_Value(RV):
    
    RV.drop(['Unnamed: 0'], axis=1, inplace=True)
    RV = RV[['home_team','away_team','half','gameId','batterName','batterId','event', 'start1B', 'start2B', 'start3B',\
             'end1B', 'end2B', 'end3B', 'startOuts','endOuts','runsFuture','runsOnPlay','outsInInning','venueId','batterPos']]
    RV['Start1'] = np.where(pd.isnull(RV['start1B']),0,1)
    RV['Start2'] = np.where(pd.isnull(RV['start2B']),0,1)
    RV['Start3'] = np.where(pd.isnull(RV['start3B']),0,1)
    RV['Start_State'] = (RV['Start1'].astype(str) + RV['Start2'].astype(str) + RV['Start3'].astype(str)+\
                          " " + RV['startOuts'].astype(str))
    RV['End1'] = np.where(pd.isnull(RV['end1B']),0,1)
    RV['End2'] = np.where(pd.isnull(RV['end2B']),0,1)
    RV['End3'] = np.where(pd.isnull(RV['end3B']),0,1)
    RV['End_State'] = (RV['End1'].astype(str) + RV['End2'].astype(str) + RV['End3'].astype(str) + \
                        " " + RV['endOuts'].astype(str))
    RV = RV[((RV.Start_State != RV.End_State) | (RV.runsOnPlay > 0)) & (RV.outsInInning == 3)]
    Start_RunExp = RV.groupby(['Start_State'])['runsFuture'].mean().reset_index().rename(columns={'runsFuture':'Start_RE'})
    RV = pd.merge(RV, Start_RunExp, on=['Start_State'], how='left')
    Base_State_3 = [pd.Series(['000 3', 0], index=Start_RunExp.columns),
                pd.Series(['001 3', 0], index=Start_RunExp.columns),
                pd.Series(['010 3', 0], index=Start_RunExp.columns),
                pd.Series(['011 3', 0], index=Start_RunExp.columns),
                pd.Series(['100 3', 0], index=Start_RunExp.columns),
                pd.Series(['101 3', 0], index=Start_RunExp.columns),
                pd.Series(['110 3', 0], index=Start_RunExp.columns),
                pd.Series(['111 3', 0], index=Start_RunExp.columns)]
    Start_RunExp = Start_RunExp.append(Base_State_3, ignore_index=True)
    End_RunExp  = Start_RunExp.rename(columns={'Start_State':'End_State', 'Start_RE':'End_RE'})
    RV = pd.merge(RV, End_RunExp, on=['End_State'], how='left')
    RV['Run_Value'] = RV['runsOnPlay'] + RV['End_RE'] - RV['Start_RE']
    
    return RV

In [4]:
RV_14 = Run_Value(MLBAM14) 
RV_15 = Run_Value(MLBAM15) 
RV_16 = Run_Value(MLBAM16) 
RV_17 = Run_Value(MLBAM17) 

C:\Users\mgy0003\AppData\Local\Temp\ipykernel_17376\2489283126.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RV['Start1'] = np.where(pd.isnull(RV['start1B']),0,1)
C:\Users\mgy0003\AppData\Local\Temp\ipykernel_17376\2489283126.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RV['Start2'] = np.where(pd.isnull(RV['start2B']),0,1)
C:\Users\mgy0003\AppData\Local\Temp\ipykernel_17376\2489283126.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

In [5]:
Event_Value14= RV_14.groupby(['event'])['Run_Value'].mean().reset_index().rename(columns = {"Run_Value": 'RV14'})
Event_Value15= RV_15.groupby(['event'])['Run_Value'].mean().reset_index().rename(columns = {"Run_Value": 'RV15'})
Event_Value16= RV_16.groupby(['event'])['Run_Value'].mean().reset_index().rename(columns = {"Run_Value": 'RV16'})
Event_Value17= RV_17.groupby(['event'])['Run_Value'].mean().reset_index().rename(columns = {"Run_Value": 'RV17'})

In [6]:
RV_Events= pd.merge(Event_Value14,Event_Value15, on = ['event'])
RV_Events= pd.merge(RV_Events,Event_Value16, on = ['event'])
RV_Events= pd.merge(RV_Events,Event_Value17, on = ['event'])
RV_Events

,event,RV14,RV15,RV16,RV17
0,Batter Interference,-0.319625,-0.363838,-0.284649,-0.430019
1,Bunt Groundout,-0.194784,-0.200346,-0.218826,-0.209411
2,Bunt Lineout,-0.303810,-0.421575,-0.352295,-0.328292
3,Bunt Pop Out,-0.316440,-0.354384,-0.342802,-0.373225
4,Catcher Interference,0.380337,0.318276,0.301623,0.399070
5,Double,0.737518,0.752039,0.743467,0.779338
6,Double Play,-0.828774,-0.854665,-0.864981,-0.897164
7,Fan interference,0.633560,0.577453,0.533316,0.590743
8,Field Error,0.462976,0.485460,0.469989,0.493206
9,Fielders Choice,0.698076,0.719351,0.701447,0.764112


In [7]:
RV_Events.corr()

,RV14,RV15,RV16,RV17
RV14,1.000000,0.998819,0.997951,0.997153
RV15,0.998819,1.000000,0.997570,0.996412
RV16,0.997951,0.997570,1.000000,0.994964
RV17,0.997153,0.996412,0.994964,1.000000


In [8]:
RV_Events['Mean'] = RV_Events.mean(axis=1)
RV_Events

C:\Users\mgy0003\AppData\Local\Temp\ipykernel_17376\3690692793.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  RV_Events['Mean'] = RV_Events.mean(axis=1)


,event,RV14,RV15,RV16,RV17,Mean
0,Batter Interference,-0.319625,-0.363838,-0.284649,-0.430019,-0.349533
1,Bunt Groundout,-0.194784,-0.200346,-0.218826,-0.209411,-0.205842
2,Bunt Lineout,-0.303810,-0.421575,-0.352295,-0.328292,-0.351493
3,Bunt Pop Out,-0.316440,-0.354384,-0.342802,-0.373225,-0.346713
4,Catcher Interference,0.380337,0.318276,0.301623,0.399070,0.349826
5,Double,0.737518,0.752039,0.743467,0.779338,0.753091
6,Double Play,-0.828774,-0.854665,-0.864981,-0.897164,-0.861396
7,Fan interference,0.633560,0.577453,0.533316,0.590743,0.583768
8,Field Error,0.462976,0.485460,0.469989,0.493206,0.477908
9,Fielders Choice,0.698076,0.719351,0.701447,0.764112,0.720747


In [9]:
RV_Events['RVSOS'] = (RV_Events['RV14'] - RV_Events['Mean'])**2 + (RV_Events['RV15'] - RV_Events['Mean'])**2 + (RV_Events['RV16'] - RV_Events['Mean'])**2 + (RV_Events['RV17'] - RV_Events['Mean'])**2

In [10]:
RV_Events

,event,RV14,RV15,RV16,RV17,Mean,RVSOS
0,Batter Interference,-0.319625,-0.363838,-0.284649,-0.430019,-0.349533,0.011787
1,Bunt Groundout,-0.194784,-0.200346,-0.218826,-0.209411,-0.205842,0.000334
2,Bunt Lineout,-0.303810,-0.421575,-0.352295,-0.328292,-0.351493,0.007724
3,Bunt Pop Out,-0.316440,-0.354384,-0.342802,-0.373225,-0.346713,0.001694
4,Catcher Interference,0.380337,0.318276,0.301623,0.399070,0.349826,0.006675
5,Double,0.737518,0.752039,0.743467,0.779338,0.753091,0.001025
6,Double Play,-0.828774,-0.854665,-0.864981,-0.897164,-0.861396,0.002402
7,Fan interference,0.633560,0.577453,0.533316,0.590743,0.583768,0.005113
8,Field Error,0.462976,0.485460,0.469989,0.493206,0.477908,0.000577
9,Fielders Choice,0.698076,0.719351,0.701447,0.764112,0.720747,0.002769


In [11]:
### Quiz 1 Work Starts Here ###
RV_Events.sort_values(by='RVSOS', ascending=False)

,event,RV14,RV15,RV16,RV17,Mean,RVSOS
28,Triple Play,-1.608127,-1.688951,-1.637543,-1.471223,-1.601461,0.025963
23,Sac Fly DP,-0.506647,-0.581165,-0.370443,-0.543715,-0.500493,0.025327
0,Batter Interference,-0.319625,-0.363838,-0.284649,-0.430019,-0.349533,0.011787
2,Bunt Lineout,-0.303810,-0.421575,-0.352295,-0.328292,-0.351493,0.007724
4,Catcher Interference,0.380337,0.318276,0.301623,0.399070,0.349826,0.006675
7,Fan interference,0.633560,0.577453,0.533316,0.590743,0.583768,0.005113
13,Grounded Into DP,-0.744986,-0.768327,-0.773318,-0.818005,-0.776159,0.002792
9,Fielders Choice,0.698076,0.719351,0.701447,0.764112,0.720747,0.002769
6,Double Play,-0.828774,-0.854665,-0.864981,-0.897164,-0.861396,0.002402
26,Strikeout - DP,-0.630963,-0.650543,-0.622787,-0.682819,-0.646778,0.002139


# II. Comparing Player Run Values

In [12]:
pd.set_option('display.max_rows', 100)

In [13]:
#Your Code Here
Player_Value14 = RV_14.groupby(['batterId','batterName'])['Run_Value'].sum().reset_index().rename(columns = {"Run_Value": 'RV14'})
Player_Value15 = RV_15.groupby(['batterId','batterName'])['Run_Value'].sum().reset_index().rename(columns = {"Run_Value": 'RV15'})
Player_Value16 = RV_16.groupby(['batterId','batterName'])['Run_Value'].sum().reset_index().rename(columns = {"Run_Value": 'RV16'})
Player_Value17 = RV_17.groupby(['batterId','batterName'])['Run_Value'].sum().reset_index().rename(columns = {"Run_Value": 'RV17'})

In [14]:
Players = pd.merge(Player_Value14,Player_Value15, on = ['batterId','batterName'])
Players = pd.merge(Players,Player_Value16, on = ['batterId','batterName'])
Players = pd.merge(Players,Player_Value17, on = ['batterId','batterName'])
Players = Players.dropna()
Players

,batterId,batterName,RV14,RV15,RV16,RV17
0,112526,Colon,-0.709186,-8.838568,-15.380841,-6.053060
1,134181,Beltre,29.004125,11.918638,29.745115,27.205050
2,136860,Beltran,-6.208773,2.355753,21.279393,-16.883042
3,150029,Werth,37.882267,-3.377653,6.368448,-7.246145
4,282332,Sabathia,-1.118087,-0.472414,-0.944580,-0.292495
...,...,...,...,...,...,...
367,608379,Wacha,-5.461352,-6.021507,-10.598557,-12.735364
368,621035,"Taylor, C",-1.686367,-8.587988,-2.211697,20.588698
369,622072,"Wood, A",-5.490464,-10.162674,-2.623027,-11.017261
370,624577,Puig,31.184967,1.223632,1.871468,5.385486


In [15]:
Players.corr()

,batterId,RV14,RV15,RV16,RV17
batterId,1.000000,-0.144210,-0.003344,-0.056668,0.091117
RV14,-0.144210,1.000000,0.466299,0.426629,0.322764
RV15,-0.003344,0.466299,1.000000,0.546136,0.510132
RV16,-0.056668,0.426629,0.546136,1.000000,0.457391
RV17,0.091117,0.322764,0.510132,0.457391,1.000000


In [16]:
import statsmodels.formula.api as smf

lmPV = smf.ols(formula = 'RV17 ~ RV14 + RV15 + RV16', data=Players).fit()
lmPV.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   RV17   R-squared:                       0.308
Model:                            OLS   Adj. R-squared:                  0.302
Method:                 Least Squares   F-statistic:                     54.61
Date:                Fri, 18 Nov 2022   Prob (F-statistic):           3.20e-29
Time:                        13:39:52   Log-Likelihood:                -1458.4
No. Observations:                 372   AIC:                             2925.
Df Residuals:                     368   BIC:                             2941.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0472      0.650      0.073      0.942      -1.231       1.326
RV14           0.0620      0.056      1.101      0.272      -0.049       0.173
RV15           0.3509      0.054      6.480      0.000       0.244       0.457
RV16           0.2673      0.059      4.532      0.000       0.151       0.383
==============================================================================
Omnibus:                       39.399   Durbin-Watson:                   1.924
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               89.833
Skew:                           0.548   Prob(JB):                     3.11e-20
Kurtosis:                       5.144   Cond. No.                         20.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

# III. Comparing Team Run Values

In [17]:
#Your Code Here
RV_14['team']= np.where(RV_14['half']=='top',RV_14['away_team'],RV_14['home_team'])
RV_15['team']= np.where(RV_15['half']=='top',RV_15['away_team'],RV_15['home_team'])
RV_16['team']= np.where(RV_16['half']=='top',RV_16['away_team'],RV_16['home_team'])
RV_17['team']= np.where(RV_17['half']=='top',RV_17['away_team'],RV_17['home_team'])

In [18]:
RVteam_14= RV_14.groupby(['team'])['Run_Value'].sum().reset_index().rename(columns= {"Run_Value": 'RV14'})
RVteam_15= RV_15.groupby(['team'])['Run_Value'].sum().reset_index().rename(columns= {"Run_Value": 'RV15'})
RVteam_16= RV_16.groupby(['team'])['Run_Value'].sum().reset_index().rename(columns= {"Run_Value": 'RV16'})
RVteam_17= RV_17.groupby(['team'])['Run_Value'].sum().reset_index().rename(columns= {"Run_Value": 'RV17'})
RVteams = pd.merge(RVteam_14,RVteam_15, on=['team'])
RVteams = pd.merge(RVteams,RVteam_16, on=['team'])
RVteams = pd.merge(RVteams,RVteam_17, on=['team'])
RVteams

,team,RV14,RV15,RV16,RV17
0,ana,104.641532,-25.056067,-3.184209,-47.579782
1,ari,-48.253012,31.924551,15.870968,62.616716
2,atl,-81.885752,-108.464134,-84.633901,-28.259905
3,bal,46.586965,29.882644,29.297940,-23.678031
4,bos,-37.383914,65.372953,160.812546,9.445347
5,cha,-1.307578,-79.136738,-41.643638,-47.416033
6,chn,-54.987532,-3.627047,85.327151,71.002092
7,cin,-57.227567,-61.401274,-17.643638,5.420218
8,cle,-3.725729,-4.178644,48.815791,73.780465
9,col,92.456063,47.127799,122.825528,78.616716


In [19]:
RVteams.corr()

,RV14,RV15,RV16,RV17
RV14,1.000000,0.363681,0.261814,0.065225
RV15,0.363681,1.000000,0.437610,0.193061
RV16,0.261814,0.437610,1.000000,0.351708
RV17,0.065225,0.193061,0.351708,1.000000


In [20]:
lmTV = smf.ols(formula = 'RV17 ~ RV14 + RV15 + RV16', data=RVteams).fit()
lmTV.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   RV17   R-squared:                       0.127
Model:                            OLS   Adj. R-squared:                  0.027
Method:                 Least Squares   F-statistic:                     1.265
Date:                Fri, 18 Nov 2022   Prob (F-statistic):              0.307
Time:                        13:43:15   Log-Likelihood:                -166.93
No. Observations:                  30   AIC:                             341.9
Df Residuals:                      26   BIC:                             347.5
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.4437     12.406     -0.036      0.972     -25.944      25.056
RV14          -0.0553      0.241     -0.230      0.820      -0.550       0.440
RV15           0.0706      0.240      0.294      0.771      -0.424       0.565
RV16           0.3788      0.231      1.638      0.113      -0.097       0.854
==============================================================================
Omnibus:                        0.584   Durbin-Watson:                   1.712
Prob(Omnibus):                  0.747   Jarque-Bera (JB):                0.061
Skew:                           0.065   Prob(JB):                        0.970
Kurtosis:                       3.179   Cond. No.                         77.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""